In [18]:
import tushare as ts
import pandas as pd
from tqdm import tqdm
import time, os
import datetime
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression
from joblib import Memory
import matplotlib.pyplot as plt
# 初始化pro接口
pro = ts.pro_api('20241217214312-98add733-f140-47dd-bfbf-26b2fa4bf60c') 
pro._DataApi__http_url = 'http://tsapi.majors.ltd:7000'

In [60]:
df = pro.index_daily(ts_code='000300.SH', start_date='20230101', end_date='20241201')
df

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,000300.SH,20241129,3916.5832,3869.8903,3962.7002,3869.8903,3872.5505,44.0327,1.1370,187656713.0,3.752620e+08
1,000300.SH,20241128,3872.5505,3904.2525,3908.3918,3866.0136,3907.0447,-34.4942,-0.8829,150117291.0,2.864329e+08
2,000300.SH,20241127,3907.0447,3829.3442,3907.1640,3817.9125,3840.1847,66.8600,1.7411,174136096.0,3.206324e+08
3,000300.SH,20241126,3840.1847,3836.2590,3875.2290,3833.2842,3848.0909,-7.9062,-0.2055,149892441.0,2.644033e+08
4,000300.SH,20241125,3848.0909,3875.6126,3889.7069,3812.3179,3865.6989,-17.6080,-0.4555,190702953.0,3.490068e+08
...,...,...,...,...,...,...,...,...,...,...,...
457,000300.SH,20230109,4013.1196,3998.5278,4023.0898,3990.7073,3980.8888,32.2308,0.8096,117054425.0,2.332124e+08
458,000300.SH,20230106,3980.8888,3969.5493,4000.4143,3966.4696,3968.5782,12.3106,0.3102,110953038.0,2.357443e+08
459,000300.SH,20230105,3968.5782,3913.4928,3974.8750,3912.2600,3892.9477,75.6305,1.9428,116502395.0,2.563126e+08
460,000300.SH,20230104,3892.9477,3886.2504,3905.9032,3873.6549,3887.8992,5.0485,0.1299,110660743.0,1.989977e+08


In [61]:
df['trade_date'] = pd.to_datetime(df['trade_date'],format='%Y%m%d')  #转化日期数据
df = df.sort_index(ascending=False)          #时间降序排列                        
df.set_index('trade_date', inplace= True)
df

,ts_code,close,open,high,low,pre_close,change,pct_chg,vol,amount
trade_date,,,,,,,,,,
2023-01-03,000300.SH,3887.8992,3864.8356,3893.9904,3831.2450,3871.6338,16.2654,0.4201,115051875.0,2.075402e+08
2023-01-04,000300.SH,3892.9477,3886.2504,3905.9032,3873.6549,3887.8992,5.0485,0.1299,110660743.0,1.989977e+08
2023-01-05,000300.SH,3968.5782,3913.4928,3974.8750,3912.2600,3892.9477,75.6305,1.9428,116502395.0,2.563126e+08
2023-01-06,000300.SH,3980.8888,3969.5493,4000.4143,3966.4696,3968.5782,12.3106,0.3102,110953038.0,2.357443e+08
2023-01-09,000300.SH,4013.1196,3998.5278,4023.0898,3990.7073,3980.8888,32.2308,0.8096,117054425.0,2.332124e+08
...,...,...,...,...,...,...,...,...,...,...
2024-11-25,000300.SH,3848.0909,3875.6126,3889.7069,3812.3179,3865.6989,-17.6080,-0.4555,190702953.0,3.490068e+08
2024-11-26,000300.SH,3840.1847,3836.2590,3875.2290,3833.2842,3848.0909,-7.9062,-0.2055,149892441.0,2.644033e+08
2024-11-27,000300.SH,3907.0447,3829.3442,3907.1640,3817.9125,3840.1847,66.8600,1.7411,174136096.0,3.206324e+08


In [62]:
df = df[['close']]
df

,close
trade_date,
2023-01-03,3887.8992
2023-01-04,3892.9477
2023-01-05,3968.5782
2023-01-06,3980.8888
2023-01-09,4013.1196
...,...
2024-11-25,3848.0909
2024-11-26,3840.1847
2024-11-27,3907.0447


In [66]:
df['close_percent'] = (df['close']/df['close'][0]-1)*100
df.to_csv('hs300_index', index = True, encoding='utf-8')

d:\temp\temp\ipykernel_24512\905021901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['close_percent'] = (df['close']/df['close'][0]-1)*100
